In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorflow.keras.utils import plot_model

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from keras.utils import np_utils

In [5]:
from tensorflow.keras.layers import concatenate 

In [6]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

In [7]:
from sklearn.utils import resample
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import dstack
from numpy import argmax

In [8]:
df= pd.read_csv("result.csv")

In [9]:
def k_fold(x,y):
    kf = KFold(n_splits = 10, shuffle = True)
    for train_index, test_index in kf.split(x):
    
        x_train, x_test = x[train_index], x[test_index] 
        y_train, y_test = y[train_index], y[test_index]

    return x_train, x_test,y_train,y_test

In [10]:
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        filename = 'models/model_' + str(i + 1) + '.h5'
        model = load_model(filename)
        all_models.append(model)
    print('>loaded %s' % filename)
    return all_models

In [11]:
def define_stacked_model(members):
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            layer.trainable = False
            layer._name = 'ensemble_' + str(i+1) + '_' + layer._name
    
    ensemble_visible = [model.input for model in members]
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense (10, activation = 'relu')(merge)
    output = Dense (6, activation = 'softmax')(hidden)
    model = Model(inputs= ensemble_visible, outputs =output)
    

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
def fit_stacked_model(model, inputX, inputy):
    X= [inputX for _ in range(len(model.input))]
    inputy_enc = np_utils.to_categorical(inputy,num_classes=6)
    return model.fit(X, inputy_enc, epochs = 100,verbose= 0)

In [13]:
def predict_stacked_model(model, inputX):
    X= [inputX for _ in range(len(model.input))]
    return model.predict(X, verbose = 0)

In [14]:
x = df.copy()
y= x.pop("clusters_pred")

In [15]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [16]:
x_train, x_test, y_train, y_test = k_fold(x_scaled, y)

In [17]:
n_members = 10
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded models/model_10.h5
Loaded 10 models


In [18]:
stacked_model = define_stacked_model(members)   

In [19]:
fit_stacked_model(stacked_model, x_test, y_test)    

In [20]:
yhat = predict_stacked_model(stacked_model, x_test)
yhat = argmax(yhat, axis=1)

In [21]:
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 0.940
